# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


 選擇
 - 數值欄位:  Age
 - 類別欄位:  Cabin

In [2]:
df.Cabin.value_counts()

B96 B98        4
C23 C25 C27    4
G6             4
F2             3
F33            3
D              3
C22 C26        3
E101           3
C83            2
E67            2
B77            2
B5             2
C92            2
E44            2
E24            2
C93            2
C126           2
D26            2
F4             2
C123           2
C52            2
B51 B53 B55    2
B20            2
B49            2
B35            2
D17            2
F G73          2
C65            2
C2             2
C125           2
              ..
D45            1
C82            1
B3             1
B73            1
E63            1
A19            1
D47            1
C45            1
E50            1
E46            1
A16            1
C30            1
C111           1
E77            1
B69            1
C99            1
C118           1
C110           1
B19            1
B80            1
B4             1
A6             1
T              1
F38            1
D6             1
C106           1
E68            1
B82 B84       

一些簡單的前處理

In [3]:
df.Embarked.fillna('None', inplace=True)
df.Cabin.fillna('None', inplace=True)
df.Age.fillna(df.Age.mean(), inplace=True)
df['Sex'] = df['Sex'].map({'male':0, 'female':1},)

In [4]:
class mode():
    def __call__(cls, x):
        return x.mode()[0]
    def __str__(cls):
        return 'mode'

In [5]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""

cabin_age_gp = df.groupby('Cabin')['Age']
new_feat = []
for func in ['mean', 'median', 'max', 'min', 'count', mode()]:
    df[f'Age_{func}'] = cabin_age_gp.transform(func)
    new_feat.append(f'Age_{func}')

df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_mean,Age_median,Age_max,Age_min,Age_count,Age_mode
0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,None,S,28.048341,29.699118,74.0,0.42,687,29.699118
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,38.000000,38.000000,38.0,38.00,1,38.000000
2,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,None,S,28.048341,29.699118,74.0,0.42,687,29.699118
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,36.000000,36.000000,37.0,35.00,2,35.000000
4,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,None,S,28.048341,29.699118,74.0,0.42,687,29.699118


In [6]:
cabin_series = df.Cabin

In [7]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

12 Numeric Features : ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Age_mean', 'Age_median', 'Age_max', 'Age_min', 'Age_count', 'Age_mode']



,Pclass,Sex,Age,SibSp,Parch,Fare,Age_mean,Age_median,Age_max,Age_min,Age_count,Age_mode
0,3,0,22.0,1,0,7.2500,28.048341,29.699118,74.0,0.42,687,29.699118
1,1,1,38.0,1,0,71.2833,38.000000,38.000000,38.0,38.00,1,38.000000
2,3,1,26.0,0,0,7.9250,28.048341,29.699118,74.0,0.42,687,29.699118
3,1,1,35.0,1,0,53.1000,36.000000,36.000000,37.0,35.00,2,35.000000
4,3,0,35.0,0,0,8.0500,28.048341,29.699118,74.0,0.42,687,29.699118


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

random_seed = 9487

In [9]:
print(f"群聚編碼欄位: {new_feat}")

群聚編碼欄位: ['Age_mean', 'Age_median', 'Age_max', 'Age_min', 'Age_count', 'Age_mode']


In [10]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

train_X = MMEncoder.fit_transform(df.drop(new_feat, axis=1))
print(f"train_X.shape: {train_X.shape}")
estimator = LogisticRegression(random_state=random_seed)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

train_X.shape: (891, 6)


0.7856545547589233

In [11]:
cross_val_score(RandomForestClassifier(random_state=random_seed), train_X, train_Y, cv=5).mean()

0.8137066381255039

In [12]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

train_X = MMEncoder.fit_transform(df)
print(f"train_X.shape: {train_X.shape}")
estimator = LogisticRegression(random_state=random_seed)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

train_X.shape: (891, 12)


0.7935008920886578

In [13]:
cross_val_score(RandomForestClassifier(random_state=random_seed), train_X, train_Y, cv=5).mean()

0.820517684491772

### 使用群聚編碼加入新的6個特徵後，增加準確度
#### 以下再研究，假使不用群聚編碼，將原本 Cabin 欄位使用傳統方式 one-hot-encoding 後準確度是怎樣

In [14]:
other_df = pd.concat(
    [
        df.drop(new_feat, axis=1),
        pd.get_dummies(cabin_series),
    ],
    axis=1
)
print(other_df.shape)
other_df.head()

(891, 154)


,Pclass,Sex,Age,SibSp,Parch,Fare,A10,A14,A16,A19,...,F E69,F G63,F G73,F2,F33,F38,F4,G6,None,T
0,3,0,22.0,1,0,7.2500,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,38.0,1,0,71.2833,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,26.0,0,0,7.9250,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,1,35.0,1,0,53.1000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,35.0,0,0,8.0500,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
train_X = MMEncoder.fit_transform(other_df)
print(f"train_X.shape: {train_X.shape}")
estimator = LogisticRegression(random_state=random_seed)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

train_X.shape: (891, 154)


0.7957480830998936

In [16]:
cross_val_score(RandomForestClassifier(random_state=random_seed), train_X, train_Y, cv=5).mean()

0.8126714180893806

以這結果來看，不做特徵篩選的情況下，one-hot 編碼比起群聚編碼共多了 142 個 feature，但減少了約 1% 的準確度